In [ ]:
# 將Google_Drive掛載到Colab的drive目錄下
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive


In [ ]:
# 將yolov4作者的原代碼複製到content目錄下
!cd /content/; git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14370, done.
remote: Total 14370 (delta 0), reused 0 (delta 0), pack-reused 14370
Receiving objects: 100% (14370/14370), 13.08 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (9768/9768), done.


In [ ]:
# 因為有使用GUP，需手動更改darknet中makefile並檢視。
%cd /content/darknet/
!sed -i "s/GPU=0/GPU=1/g" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/g" Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" Makefile
!head Makefile

/content/darknet
GPU=1
CUDNN=1
CUDNN_HALF=1
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0



In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

In [ ]:
!cp -r /drive/My\ Drive/Colab\ Notebooks/data /content/darknet/data/
!cd /content/darknet/data/data; 7z x test.7z; 7z x test_FinalData.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 350070 bytes (342 KiB)

Extracting archive: test.7z
--
Path = test.7z
Type = 7z
Physical Size = 350070
Headers Size = 602
Method = LZMA2:384k
Solid = +
Blocks = 1

  0%    Everything is Ok

Files: 36
Size:       360903
Compressed: 350070

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 32356978 bytes (31 MiB)

Extracting archive: test_FinalData.7z
--
Path = test_FinalData.7z
Type = 7z
Physical Size = 32356978
Headers Size = 27814
Method = LZMA2:25
Solid = +
Blocks = 1

  0%     

In [ ]:
%cd /content/darknet/data/
!mkdir yolo
!mkdir img
!mkdir lab

/content/darknet/data


In [ ]:
# 將jpg檔與txt檔分發到images與labels資料夾中。
import os
import glob
import shutil

labels = []
path = "/content/darknet/data/data"
for dir in os.listdir(f'{path}/'):
  img = 0; lab = 0
  if dir.find('total') != -1:
    labels.append(dir[6:])

    for item in glob.glob(f'{path}/{dir}/*'):
      if item.find('class') != -1: os.remove(item)
      else:
        if item.find('.jpg') != -1:
          shutil.copy(item,'/content/darknet/data/img/')
          img += 1
        else: 
          shutil.copy(item,'/content/darknet/data/lab/')
          lab += 1
      
    print(f'copy {dir[6:]} img {img} num to images.')
    print(f'copy {dir[6:]} img {lab} num to images.')
    print('-------------------------------------')

img_num = []; lab_num = []
for i in os.listdir('/content/darknet/data/img/'):
  img_num.append(i)
for i in os.listdir('/content/darknet/data/lab/'):
  lab_num.append(i)
print(f'total img num:{len(img_num)}')
print(f'total lab num:{len(lab_num)}')

copy eggplant img 250 num to images.
copy eggplant img 250 num to images.
-------------------------------------
copy egg img 250 num to images.
copy egg img 250 num to images.
-------------------------------------
copy cabbage img 250 num to images.
copy cabbage img 250 num to images.
-------------------------------------
copy banana img 250 num to images.
copy banana img 250 num to images.
-------------------------------------
copy tomato img 250 num to images.
copy tomato img 250 num to images.
-------------------------------------
copy onion img 250 num to images.
copy onion img 250 num to images.
-------------------------------------
total img num:1500
total lab num:1500


In [ ]:
# 創建obj.name檔(順序可能會亂，需手動調整)
with open('/content/darknet/cfg/obj.names','w')as file:
  file.write('\n'.join(labels))
print(f'write_in {len(labels)} labels: {", ".join(labels)}') 

write_in 6 labels: eggplant, egg, cabbage, banana, tomato, onion


In [ ]:
# 此處obj.names中的類別名稱的順序如果顯示非如下，則請直接點擊該檔案修改存檔。
# banana
# cabbage
# egg
# eggplant
# onion
# tomato
with open('/content/darknet/cfg/obj.names','r') as f:
  f_content = f.read()
  print(f_content)

banana
cabbage
egg
eggplant
onion
tomato


In [ ]:
# 移除標註錯誤文檔(15與17)
error_num = []; new_data = ""
lab_path = '/content/darknet/data/lab/'
img_path = '/content/darknet/data/img/'
for item in os.listdir(lab_path):
  with open(lab_path+item,'r') as f:
    data = f.read()
    if '15 ' in data or '17 ' in data: 
      # os.remove(lab_path+item)
      error_num.append(item)
    elif '6' in data:
      new_data = data.replace('6 ','5 ')
  with open(lab_path+item,'w') as f:
    f.write(new_data)
print(f'all error labels num:{len(error_num)}')


# 移除錯誤圖檔
cancel = 0
for item in os.listdir(img_path):
  for ero in error_num:
    if item[:-4] == ero[:-4]:
      os.remove(img_path+item)
      os.remove(lab_path+item[:-4]+'.txt')
      cancel += 1
print(f'cancel num of lab and img:{cancel}')

img_num = []; lab_num = []
for i in os.listdir('/content/darknet/data/img/'):
  img_num.append(i)
for j in os.listdir('/content/darknet/data/lab/'):
  lab_num.append(j)

print(f'total img num: {len(img_num)}')
print(f'total lab num: {len(lab_num)}')

all error labels num:200
cancel num of lab and img:200
total img num: 1300
total lab num: 1300


In [ ]:
# 將jpg與txt全部移動到yolo資料夾中(此資料夾是obj.data的主要路徑)。
num_img = 0; num_txt = 0
for item in glob.glob('/content/darknet/data/lab/*'):
  shutil.copy(item,'/content/darknet/data/yolo/')
  num_txt += 1
for item in glob.glob('/content/darknet/data/img/*'):
  shutil.copy(item,'/content/darknet/data/yolo/')
  num_img += 1

print(f'yolo dir num of img: {num_img}')
print(f'yolo dir num of img: {num_txt}')

yolo dir num of img: 1300
yolo dir num of img: 1300


In [ ]:
# 1.將yolo資料夾中的txt檔做random與split。
# 2.以處理好的txt檔為對照，將同名稱jpg檔的"路徑+檔名"複製到train與test中。

import random
import glob
import os

cfg_path = '/content/darknet/cfg'
yolo_path = '/content/darknet/data/yolo'
txt_path = glob.glob(f'{yolo_path}/*.txt')
random.shuffle(txt_path)
num_train_images = int(len(txt_path)*0.8)

with open(f"{cfg_path}/train.txt", 'w') as f:
  for path in txt_path[:num_train_images]:
    path = os.path.basename(path).replace(".txt",".jpg")
    if path: f.write(f'{yolo_path}/{path}\n')
with open(f"{cfg_path}/test.txt", 'w') as f:
  for path in txt_path[num_train_images:]:
    path = os.path.basename(path).replace(".txt",".jpg")
    if path: f.write(f'{yolo_path}/{path}\n')

In [ ]:
# 1.建立obj.data檔(此檔為模型訓練與測試時個檔案的路徑指示)
# 2.類別數量請手動修改。 
with open("/content/darknet/cfg/obj.data", 'w') as f:
  f.write("classes=6\n")
  f.write("train=/content/darknet/cfg/train.txt\n")
  f.write("valid=/content/darknet/cfg/test.txt\n")
  f.write("names=/content/darknet/cfg/obj.names\n")
  f.write("backup=/content/darknet/backup\n")
with open("/content/darknet/cfg/obj.data", 'r') as f:
  print(f.read())

classes=6
train=/content/darknet/cfg/train.txt
valid=/content/darknet/cfg/test.txt
names=/content/darknet/cfg/obj.names
backup=/content/darknet/backup



In [ ]:
# 手動更改cfg檔中的設定並檢視。
# 主要更動filter與class_num，細部請上網搜尋。
%cd /content/darknet/cfg/
!mv yolov4-tiny-custom.cfg yolov4-tiny-obj.cfg
!sed -i '212s/255/33/' yolov4-tiny-obj.cfg
!sed -i '220s/80/6/' yolov4-tiny-obj.cfg
!sed -i '263s/255/33/' yolov4-tiny-obj.cfg
!sed -i '269s/80/6/' yolov4-tiny-obj.cfg
!sed -n -e 212p -e 220p -e 263p -e 269p yolov4-tiny-obj.cfg

/content/darknet/cfg
filters=33
classes=6
filters=33
classes=6


In [ ]:
%cd /content/darknet/
! ./darknet detector calc_anchors cfg/obj.data -num_of_clusters 9 -width 416 -height 416 -showpause

/content/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 416, height = 416 
 read labels from 1040 images 
 loaded 	 image: 1040 	 box: 1586
 all loaded. 

 calculating k-means++ ...

 iterations = 22 


counters_per_class = 253, 197, 206, 280, 288, 362

 avg IoU = 78.87 % 

Saving anchors to the file: anchors.txt 
anchors =  67,102, 129, 88, 111,157,  94,287, 167,216, 286,160, 214,308, 309,266, 347,344



In [ ]:
%cd /drive/My\ Drive/Colab\ Notebooks/cfg/
!chmod -R 777 /drive/My\ Drive/Colab\ Notebooks/cfg
!cp yolov4-tiny.conv.29 /content/darknet/ 

/drive/My Drive/Colab Notebooks/cfg


In [ ]:
%cd /content/darknet/
!./darknet detector train cfg/obj.data cfg/yolov4-tiny-obj.cfg ./yolov4-tiny.conv.29 -dont_show -map

串流輸出內容已截斷至最後 5000 行。
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 30 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 62, class_loss = 1.937500, iou_loss = 0.000012, total_loss = 1.937512 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 37 Avg (IOU: 0.000000, GIOU: -0.000617), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 41, class_loss = 1.281250, iou_loss = 0.000003, total_loss = 1.281253 
 total_bbox = 376573, rewritten_bbox = 0.019385 % 


 Tensor Cores are used.
 (next mAP calculation at 4000 iterations) 
 3909: 1.610469, 1.532797 avg loss, 0.003000 rate, 0.624165 seconds, 250176 images, 1.540444 hours left
Loaded: 0.000039 seconds
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 30 Avg (IOU: 0.000000, GIOU: -0.000191), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 70, class_loss = 2.187500, iou_lo

In [ ]:
%cd /content/darknet/
!./darknet detector test cfg/obj.data cfg/yolov4-tiny-obj.cfg backup/yolov4-tiny-obj_best.weights /content/darknet/data/data/2731.jpg -dont_show

/content/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x

In [ ]:
%cd /drive/My\ Drive/Colab\ Notebooks/
!cp /content/darknet/darknet .
!cp /content/darknet/backup/yolov4-tiny-obj_best.weights .
!cp /content/darknet/chart.png .
!cp /content/darknet/cfg/yolov4-tiny-obj.cfg .
!cp /content/darknet/yolov4-tiny.conv.29 .
!cp /content/darknet/cfg/obj.data .
!cp /content/darknet/cfg/obj.names .

/drive/My Drive/Colab Notebooks
